In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

In [3]:
fake = pd.read_csv("C:/Users/sreya/Downloads/archive (1)/Fake.csv")
true = pd.read_csv("C:/Users/sreya/Downloads/archive (1)/True.csv")

In [4]:
fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [8]:
print(len(fake),len(true)) # to check the length of the dataset

23481 21417


In [11]:
import re
try:
    import nltk
    nltk.data.find("tokenizers/punkt")
except Exception:
    print("Downloading minimal NLTK data (punkt, stopwords). This may require internet access...")
import nltk
nltk.download('punkt')
nltk.download('stopwords')


from nltk.corpus import stopwords
_stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sreya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sreya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [13]:
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return
# return new lines      
    text = text.replace('\n', ' ').replace('\r', ' ')
# remove URLs
    text = re.sub(r'http\S+|www\.\S+', '', text)
# keep basic punctuation, remove strange characters
    text = re.sub(r"[^A-Za-z0-9\.,!?:;\'\"\s-]", ' ', text)
# collapse whitespace
    text = re.sub(r"\s+", ' ', text).strip()
    return text

In [14]:
# Combine title and text when available
fake['content'] = (fake.get('title', '').fillna('') + '. ' + fake.get('text', '').fillna('')).apply(clean_text)
true['content'] = (true.get('title', '').fillna('') + '. ' + true.get('text', '').fillna('')).apply(clean_text)

In [15]:
# dataframe with labels: fake=1, real=0
fake = fake.assign(label=1)
true = true.assign(label=0)

In [16]:
# merge the two dataframes of fake and true news and shuffle the rows
df = pd.concat([fake[['content','label']], true[['content','label']]], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [17]:
df.head()

,content,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,1
1,Trump drops Steve Bannon from National Securit...,0
2,Puerto Rico expects U.S. to lift Jones Act shi...,0
3,OOPS: Trump Just Accidentally Confirmed He Lea...,1
4,Donald Trump heads for Scotland to reopen a go...,0


In [18]:
# drop empty columns
df = df[df['content'].str.strip() != ''].reset_index(drop=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44889 entries, 0 to 44888
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  44889 non-null  object
 1   label    44889 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 701.5+ KB


In [23]:
print(f"Combined dataset shape: {df.shape}")
print(df['label'].value_counts())

Combined dataset shape: (44889, 2)
label
1    23472
0    21417
Name: count, dtype: int64


In [24]:
print(df.loc[0,'content'][0:]) 

Ben Stein Calls Out 9th Circuit Court: Committed a Coup d tat Against the Constitution. 21st Century Wire says Ben Stein, reputable professor from, Pepperdine University also of some Hollywood fame appearing in TV shows and films such as Ferris Bueller s Day Off made some provocative statements on Judge Jeanine Pirro s show recently. While discussing the halt that was imposed on President Trump s Executive Order on travel. Stein referred to the judgement by the 9th Circuit Court in Washington state as a Coup d tat against the executive branch and against the constitution. Stein went on to call the Judges in Seattle political puppets and the judiciary political pawns. Watch the interview below for the complete statements and note the stark contrast to the rhetoric of the leftist media and pundits who neglect to note that no court has ever blocked any Presidential orders in immigration in the past or discuss the legal efficacy of the halt or the actual text of the Executive Order.READ MO

## Modelling

In [53]:
# Purpose: Create a reproducible split and save to artifacts/ for later steps
from sklearn.model_selection import train_test_split
OUT = Path("artifacts")
OUT.mkdir(exist_ok=True)

train_df, test_df = train_test_split(df, test_size=0.20, stratify=df['label'], random_state=42)
train_df.to_csv(OUT / 'train.csv', index=False)
test_df.to_csv(OUT / 'test.csv', index=False)
print(f"Saved train ({len(train_df)}) and test ({len(test_df)}) to {OUT}")

Saved train (35911) and test (8978) to artifacts


In [54]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import joblib


X_train = train_df['content'].tolist()
y_train = train_df['label'].values
X_test = test_df['content'].tolist()
y_test = test_df['label'].values


baseline_pipe = Pipeline([
('tfidf', TfidfVectorizer(max_features=20000, ngram_range=(1,2))),
('clf', LogisticRegression(max_iter=1000))
])


print("Training baseline model (TF-IDF + LogisticRegression)...")
baseline_pipe.fit(X_train, y_train)
print("Training complete.")


preds = baseline_pipe.predict(X_test)
print('\nClassification report:')
print(classification_report(y_test, preds, digits=4))
print('\nConfusion matrix:')
print(confusion_matrix(y_test, preds))


# save pipeline
joblib.dump(baseline_pipe, OUT / 'baseline_tfidf_logreg.joblib')
print(f"Saved baseline pipeline to {OUT / 'baseline_tfidf_logreg.joblib'}")

Training baseline model (TF-IDF + LogisticRegression)...
Training complete.

Classification report:
              precision    recall  f1-score   support

           0     0.9873    0.9946    0.9909      4283
           1     0.9951    0.9883    0.9917      4695

    accuracy                         0.9913      8978
   macro avg     0.9912    0.9915    0.9913      8978
weighted avg     0.9913    0.9913    0.9913      8978


Confusion matrix:
[[4260   23]
 [  55 4640]]
Saved baseline pipeline to artifacts\baseline_tfidf_logreg.joblib


In [55]:
# Purpose: get token-level contributions quickly without LIME

# Extract vectorizer and classifier from pipeline
vectorizer = baseline_pipe.named_steps['tfidf']
clf = baseline_pipe.named_steps['clf']
feature_names = vectorizer.get_feature_names_out()

def explain_instance_fast(text: str, num_features: int = 8):
    """Return top contributing tokens using logistic regression coefficients."""
    X_vec = vectorizer.transform([text])
    contrib = X_vec.multiply(clf.coef_[0]).toarray()[0]
    top_idx = contrib.argsort()[-num_features:][::-1]
    features = [(feature_names[i], contrib[i]) for i in top_idx]
    probs = baseline_pipe.predict_proba([text])[0]
    return features, probs

# Quick demo
sample_text = X_test[0]
features, probs = explain_instance_fast(sample_text, num_features=6)
print("Model probs (real,fake):", probs)
print("Top features and weights:")
for feat, w in features:
    print(f" {feat} -> {w:+.4f}")

Model probs (real,fake): [0.00224196 0.99775804]
Top features and weights:
 obama -> +0.8346
 the -> +0.7732
 our -> +0.5434
 that -> +0.4682
 is -> +0.4596
 this -> +0.3725


In [58]:
# Generative justification using Hugging Face (FLAN-T5 small, PyTorch)
# ----------------------------
# Purpose: Use a small text-to-text model to convert evidence + prediction into a justification.
# NOTE: Only requires `transformers` + `torch`, TensorFlow/Keras not needed.

import torch
from transformers import pipeline

MODEL_NAME = "google/flan-t5-small"  # compact model for demo
device = 0 if torch.cuda.is_available() else -1
print("Generator device:", "GPU" if device == 0 else "CPU")

# Force PyTorch-only backend
generator = pipeline(
    "text2text-generation",
    model=MODEL_NAME,
    device=device,
    framework="pt"
)

def make_prompt(article_text: str, label_str: str, confidence: float, feats: list):
    """Create a concise prompt that includes evidence tokens and asks for a justification."""
    feat_lines = "\n".join([f"- '{t[0]}' ({t[1]:+.3f})" for t in feats]) if feats else "No salient tokens."
    prompt = (
        f"Article: {article_text}\n"
        f"Prediction: {label_str} (confidence {confidence:.2f})\n"
        f"Top contributing tokens:\n{feat_lines}\n\n"
        "Task: Write a short (2-3 sentence) justification explaining why the model labeled this article as "
        f"{label_str.upper()}, using the tokens above as evidence."
    )
    return prompt

def generate_justification(article_text: str):
    """Predict Fake/Real and generate a justification with evidence tokens."""
    probs = baseline_pipe.predict_proba([article_text])[0]
    pred = baseline_pipe.predict([article_text])[0]
    label_str = "fake" if pred == 1 else "real"
    feats, _ = explain_instance_fast(article_text, num_features=6)  # <-- faster version
    prompt = make_prompt(article_text, label_str, max(probs), feats)
    out = generator(prompt, max_length=120, do_sample=False)[0]["generated_text"]
    return {
        "label": label_str,
        "confidence": float(max(probs)),
        "features": feats,
        "prompt": prompt,
        "justification": out,
    }

# Demo
print("\nGenerating justification for a sample test article...")
res = generate_justification(sample_text)
print("PREDICTION:", res["label"], "| CONF:", res["confidence"])
print("\nJUSTIFICATION:\n", res["justification"])


Generator device: CPU


Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (2249 > 512). Running this sequence through the model will result in indexing errors



Generating justification for a sample test article...


Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


PREDICTION: fake | CONF: 0.9977580446745059

JUSTIFICATION:
 This is the most brilliant and scary analysis of where we, as a nation are today Last year a retired Border Patrol Officer by the name of Zach Taylor went on camera to explain the driving force behind the unprecedented surge in illegal immigration happening on our southern border. Taylor went on to note that what was happening at our border was not due to a spur of the moment event, or a humanitarian crisis, but asymmetrical warfare. The surge we saw at the border was apart of a larger more chilling event that served one purpose and one purpose only, to show our enemies that our southern border had been compromised and the government wouldn t do a damn thing about it.The surge we saw at the border was apart of a larger more chilling event that served one purpose and one purpose only, to show our enemies that our southern border had been compromised and the government wouldn t do a damn thing about it.The surge we saw at the b

In [62]:
# ----------------------------
# Cell 7 - Batch generate & save justifications
# ----------------------------
OUT_JUST = OUT / "justifications.csv"

rows = []
for i in range(min(50, len(X_test))):  # limit to 50 for speed
    txt = X_test[i]
    r = generate_justification(txt)
    rows.append({
        "content": txt,
        "pred": r["label"],
        "confidence": r["confidence"],
        "justification": r["justification"],
        "features": str(r["features"])
    })

pd.DataFrame(rows).to_csv(OUT_JUST, index=False, encoding="utf-8")
print(f"Wrote {len(rows)} justifications to {OUT_JUST}")


Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Wrote 50 justifications to artifacts\justifications.csv


### Model Deployment in streamlit

In [68]:
# ----------------------------
# Cell 8 - Streamlit app
# ----------------------------
streamlit_app = r"""
import streamlit as st
import joblib
from transformers import pipeline
import torch

# Load baseline model
model = joblib.load("artifacts/baseline_tfidf_logreg.joblib")
vectorizer = model.named_steps["tfidf"]
clf = model.named_steps["clf"]
feature_names = vectorizer.get_feature_names_out()

def explain_instance_fast(text: str, num_features: int = 6):
    X_vec = vectorizer.transform([text])
    contrib = X_vec.multiply(clf.coef_[0]).toarray()[0]
    top_idx = contrib.argsort()[-num_features:][::-1]
    return [(feature_names[i], contrib[i]) for i in top_idx], model.predict_proba([text])[0]

# PyTorch-only generator
device = 0 if torch.cuda.is_available() else -1
generator = pipeline("text2text-generation", model="google/flan-t5-small", device=device, framework="pt")

def generate_justification(text: str):
    probs = model.predict_proba([text])[0]
    pred = model.predict([text])[0]
    label = "fake" if pred == 1 else "real"
    feats, _ = explain_instance_fast(text, 6)
    feat_lines = "\\n".join([f"- {f} ({w:+.3f})" for f,w in feats])
    prompt = f"Article: {text}\\nPrediction: {label} (conf {max(probs):.2f})\\nFeatures:\\n{feat_lines}\\nWrite a short justification."
    out = generator(prompt, max_new_tokens=120, do_sample=False)[0]["generated_text"]
    return label, out, feats, probs

# Streamlit UI
st.title("📰 Fake News Detector + Justification")
text = st.text_area("Paste a news article here:")

if st.button("Check") and text.strip():
    label, justification, feats, probs = generate_justification(text)
    st.write(f"Prediction: **{label.upper()}** (Confidence: {max(probs):.2f})")
    st.subheader("Justification")
    st.write(justification)
    st.subheader("Top Features")
    for f,w in feats:
        st.write(f"- {f} ({w:+.3f})")
"""

# Write to app.py
with open("app.py", "w", encoding="utf-8") as f:
    f.write(streamlit_app)

print("✅ Streamlit app written to app.py. Run with: streamlit run app.py")


✅ Streamlit app written to app.py. Run with: streamlit run app.py


In [69]:
!dir app.py 

 Volume in drive C is YUMI
 Volume Serial Number is 7A88-5D93

 Directory of C:\Users\sreya

21-09-2025  05:21             1,821 app.py
               1 File(s)          1,821 bytes
               0 Dir(s)  173,602,611,200 bytes free
